<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Complete_Pheme_Veracity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import fastai

In [3]:
from fastai.text import * 

In [4]:
def read(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [5]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [6]:
# For veracity
train = read_veracity('train.csv', True)
test = read_veracity('test.csv')

In [7]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [8]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [9]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [10]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [11]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [12]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.574623,4.279481,0.289414,02:07


In [13]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.073269,3.974274,0.327204,02:36


In [14]:
learn.predict("This is a review about", n_words=10)

'This is a review about Charlie Hebdo : \n  http : / /'

In [15]:
learn.save_encoder('ft_enc')

In [16]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (55924 items)
x: TextList
xxbos @tariqnasheed @mike_t_lopez i can show you hundreds of cases of black violence against whites vs a few of whites vs blacks . xxmaj and 10 times ?,xxbos @rt_com xxmaj ca n't get through to daughter & & grand - kids in central xxmaj sydney . xxmaj just have to hope they 're fine .,xxbos @mysteryvlog hahahahaha !,xxbos @k_ur_crazy xxmaj yo , just tell me who that honey is in your avatar so i can go occupy myself tryna get a date with her fine self , aw xxunk,xxbos xxup sydney xxup terror 
  - xxmaj heavily - armed police storm cafe 
  - xxmaj multiple hostages flee 
  - xxmaj hostages seriously wounded 
  - xxmaj gunman said to be dead
y: CategoryList
False,True,False,False,False
Path: .;

Valid: LabelList (13982 items)
x: TextList
xxbos @allierenison it 's a serious question . xxmaj she was listed in his manifesto .,xxbos @mailonline @ukrpravda_news xxunk бы xxunk просто xxunk .,xxbos @cbcalerts,xxbos xx

In [17]:
data_clas.show_batch()

text,target
xxbos .@lisabloom wear your best i ❤ ️ xxmaj surveillance tee \n xxup interntnl xxup wave xxup at xxup surveillance xxup day tomorrow 🙋 \n http : / / t.co / lvnpqfjuw9 \n @till_you @lisabloom xxmaj you did n't get any facts or proof as all this would prove is that he stole cigars . xxmaj has no relation on the death . xxunk \n @androoshaw,True
"xxbos @bttoronto xxup rip , i , xxup ask xxup of xxup god , xxup in xxup thy xxup name xxup of xxup jesus & & i , xxup say xxup to xxup his xxup family xxup to xxup stay , xxup stand , xxup strong - xxup xxunk xxup xxunk xxup sympathy & & xxup xxunk",True
xxbos xxup @ap : xxmaj excuse xxmaj me ... xxup but there is a # xxunk xxmaj also that has been plastered on the xxmaj news ! ! xxmaj do xxup you all xxup not xxup have xxup that xxup info ? # xxmaj suspects # xxup really 👀,False
xxbos @to_sportsgirl xxup please xxup listen xxup we xxup need \n xxup world xxup peace \n http : / / t.co / hjvy8bcny9 \n 🔥 xxup world xxup peace 💣 \n xxmaj jahmal xxmaj brown ft . xxmaj jahyda \n https : / / t.co / 9jqfgnn1kn,True
"xxbos @rt_com : xxup schools xxup closed , xxup dammartin - xxup en - xxup goele xxup residents xxup told xxup to xxup stay xxup xxunk xxup like xxup warzone ' http : / / t.co / xxunk http : / / t.co / xxunk",True


In [18]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.548666,0.520131,0.743456,03:11


In [19]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.493396,0.461409,0.784866,03:08


In [20]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.469428,0.437241,0.797811,05:31
1,0.457169,0.402721,0.816049,05:29
2,0.409481,0.363904,0.831998,05:45
3,0.385285,0.320202,0.856530,05:29
4,0.334313,0.298845,0.865971,05:16
5,0.294113,0.286327,0.871263,05:15
6,0.237396,0.285627,0.875483,05:19
7,0.206850,0.288578,0.875197,05:32
8,0.180293,0.306205,0.874124,05:33
9,0.188670,0.306616,0.874124,05:29


In [21]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [22]:
from sklearn.metrics import accuracy_score, f1_score

In [23]:
accuracy_score(test['label'], l)

0.8723071808510638

In [24]:
f1_score(test['label'], l, average='macro')

0.8428953782530009

In [25]:
learn.export('veracity.pkl')